Transform video to audio

In [ ]:
from audio_extractor import extract_audio

debra_video = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Soundtrack/[ONTIVA.COM] Debra-144p.mp4"
debra_audio_file = "debra_song"

baby_driver_video_file = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Baby.Driver.2017.720p.BluRay.x264-[YTS.AG].mp4"
baby_driver_audio_file = "baby_driver_audio"

sample_rate = 22000

extract_audio(debra_video, sample_rate, debra_audio_file)
extract_audio(baby_driver_video_file, sample_rate, baby_driver_audio_file)

Generate Descriptors

In [12]:
from audio_descriptor_generator import create_audio_descriptors

sample_rate = 22000
descriptors_per_second = 10
dimension = 32
window = int(sample_rate / descriptors_per_second)
hop = window

baby_driver_audio_file = "baby_driver_audio.22000.wav"
debra_song_audio_file = "debra_song.22000.wav"

debra_descriptors = create_audio_descriptors(debra_song_audio_file, sample_rate, dimension, window, hop)
print(debra_descriptors.shape)
baby_descriptors = create_audio_descriptors(baby_driver_audio_file, sample_rate, dimension, window, hop)
print(baby_descriptors.shape)

baby_driver_descriptors_file = f"baby_driver_audio_descriptors_{descriptors_per_second}.bin"
debra_song_descriptors_file = f"debra_song_descriptors_{descriptors_per_second}.bin"

debra_descriptors.tofile(debra_song_descriptors_file, sep="\n")
baby_descriptors.tofile(baby_driver_descriptors_file, sep="\n")

song_shape = debra_descriptors.shape
movie_shape = baby_descriptors.shape

(3434, 32)
(67598, 32)


Calculate distances

In [13]:
import numpy
import time
from scipy.spatial import distance
"""
descriptors_per_second = 10
song_shape = (687, 32)
movie_shape = (13520, 32)
"""
baby_driver_descriptors_file = f"baby_driver_audio_descriptors_{descriptors_per_second}.bin"
# debra_song_descriptors_file = "debra_song_descriptors.bin"
debra_song_descriptors_file = f"debra_song_descriptors_{descriptors_per_second}.bin"

debra_song_shape = song_shape

song_descriptors = numpy.fromfile(debra_song_descriptors_file, sep="\n").reshape(debra_song_shape)
print(song_descriptors.shape)

movie_shape = movie_shape
movie_descriptors = numpy.fromfile(baby_driver_descriptors_file, sep="\n").reshape(movie_shape)
print(movie_descriptors.shape)

t0 = time.time()
distances = distance.cdist(song_descriptors, movie_descriptors)
# distances = distance.cdist(movie_descriptors, movie_descriptors)
t1 = time.time()

print(f"Distances {distances.shape} {round(t1-t0, 2)} secs")

(3434, 32)
(67598, 32)
Distances (3434, 67598) 12.84 secs


Get neighbours

In [26]:
from duplicate_searcher import Neighbours

number_of_neighbours = 10

neighbour_file = "neighbours.txt"
neighbours = []
total = distances.shape[0]
t10 = time.time()
for i in range(distances.shape[0]):
    print(f"Progress: {round((i/total)*100, 2)}%", end="\r")
    song_descriptor = distances[i]
    neighs = numpy.argpartition(song_descriptor, number_of_neighbours)[:number_of_neighbours]
    neighbours.append(neighs)
t11 = time.time()
neighbours = Neighbours(numpy.array(neighbours))
print(f"Neighbours {neighbours.shape()} {round(t11-t10, 2)} secs")

Neighbours (3434, 10) 6.69 secs


Create Candidates

In [27]:
from duplicate_searcher import Candidate

candidates = []
song_indexes = range(distances.shape[0])
total_songs = distances.shape[0]
for song in song_indexes:
    print(f"Progress {round((song/total_songs)*100, 2)}%", end="\r")
    for neighbour in neighbours.search(song):
        candidates.append(Candidate(song, neighbour, descriptors_per_second))

print(f" Candidates {len(candidates)}")


 Candidates 34340


Find sequence

In [29]:
copies = []

max_missing_streak_secs = 1
max_missing_streak = max_missing_streak_secs * descriptors_per_second
min_duration_seconds = 2
min_duration = min_duration_seconds * descriptors_per_second

total_candidates = len(candidates)
for i in range(len(candidates)):
    print(f"Progress {round((i/total_candidates)*100, 2)}%", end="\r")
    cand = candidates[i]
    current_candidate = cand.find_next(neighbours, max_missing_streak)
    if current_candidate.sequence_duration >= min_duration and current_candidate.score() >= 1:
        copies.append(current_candidate)

print(len(copies))   
    

84


Contain

In [30]:
filtered = []

for i in range(len(copies)):
    print(f"Progress {round((i/len(copies))*100, 2)}%", end="\r")
    cani = copies[i]
    add = True
    for j in range(len(copies)):
        canj = copies[j]
        
        if i == j:
            continue
        
        elif cani.contains(canj):
            add = False
            break
    
    if add:
        filtered.append(cani)

print(len(filtered))


40


Sort

In [31]:
# sorted_candidates = filtered
sorted_candidates = sorted(filtered, key=lambda c: c.song_descriptor_index)

print(len(sorted_candidates))

40


Combine

In [32]:
max_offset_secs = 2
max_offset = max_offset_secs * descriptors_per_second
max_combine_dist_secs = 3
max_combine_distance = max_combine_dist_secs * descriptors_per_second

for i in range(len(sorted_candidates)):
    copy_i = sorted_candidates[i]
    for j in range(i + 1, len(sorted_candidates)):
        copy_j = sorted_candidates[j]

        if (copy_i.distance(copy_j) <= max_combine_distance) and (copy_i.offset_diff(copy_j) <= max_offset):
            copy_i.combine(copy_j)

print(len(sorted_candidates))

40


Overlapped

In [33]:
repeated = set()
off_set_diff_limit_secs = 10
off_set_diff_limit = off_set_diff_limit_secs * descriptors_per_second
max_overlap_dist_secs = 3
max_overlap_distance = max_overlap_dist_secs * descriptors_per_second

for i in range(len(sorted_candidates)):
    copy_i = sorted_candidates[i]
    for j in range(i + 1, len(sorted_candidates)):
        copy_j = sorted_candidates[j]

        if copy_i.distance(copy_j) <= max_overlap_distance and copy_i.offset_diff(copy_j) <= off_set_diff_limit_secs:
            if copy_i.sequence_duration >= copy_j.sequence_duration:
                repeated.add(copy_j)
            else:
                repeated.add(copy_i)

for copy in repeated:
    sorted_candidates.remove(copy)

print(len(sorted_candidates))

1


Delete short copies

In [34]:
filtered_copies = []

for copy in sorted_candidates:
    if copy.sequence_duration > min_duration:
        filtered_copies.append(copy)

print(len(filtered_copies))

1


Show Sequences

In [41]:
import datetime

print("Song secs | Movie secs")

for match in filtered_copies:
    print(f"{match.song_descriptor_index} - {match.song_end_index()} | {match.movie_descriptor_index} - {match.movie_end_index()}")
    print(f"{str(datetime.timedelta(seconds=int(match.song_descriptor_index / descriptors_per_second)))} - {str(datetime.timedelta(seconds=int(match.song_end_index() / descriptors_per_second)))} | {str(datetime.timedelta(seconds=int(match.song_descriptor_index / descriptors_per_second)))} - {str(datetime.timedelta(seconds=int(match.movie_end_index() / descriptors_per_second)))}")

Song secs | Movie secs
3359 - 3421 | 67550 - 67612
0:05:35 - 0:05:42 | 0:05:35 - 1:52:41
